# Run Active Filaments Simulations
Setup and run an active-filament simulation parametric sweep (using parallelization) and save the data.


In [1]:
import sys
if 'init_modules' in globals().keys():
    # second or subsequent run: remove all but initially loaded modules
    for m in sys.modules.keys():
        if m not in init_modules:
            del(sys.modules[m])
else:
    # first run: find out which modules were initially loaded
    init_modules = sys.modules.keys()
    
import numpy as np
from scipy import signal
from scipy import interpolate
import matplotlib.pyplot as plt 
from sys import platform
from joblib import Parallel, delayed
import multiprocessing
from tqdm import tqdm

from pyfilaments.activeFilaments import activeFilament
from pyfilaments._def import *

linux system


In [2]:
# Function definitions

def run_parametric_simulation(parameter_to_vary, parameter_value, activity_parameters, 
                              Tf, Npts, spatial_activity_type):
    
    # Build a key-word dictionary that can be passed to the Active filament class
    kwargs = {'dim':DIMS, 'Np':NP,'radius':RADIUS, 'b0':B0, 'k':K, 'S0':S0, 'D0':D0, 'bc':BC}
    
    if parameter_to_vary == 'activity time scale':
        # If the activity time-scale is changing, recalculate some variables
        activity_timescale = parameter_value
        Tf = activity_timescale*N_CYCLES
        Npts = int(Tf/DELTA_T) # No:of time points saved
        activity_parameters[parameter_to_vary] = parameter_value
        
    else:
        # Set the parameter that is being varied (non activity time scale related parameters)
        kwargs[parameter_to_vary] = parameter_value
        
        
    # Initialize the activeFilament class based on these parameter values
    active_filament = activeFilament(**kwargs)
    
    # Run the filament simulation
    active_filament.simulate(Tf, Npts,  save = True, overwrite = False, 
                            path = ROOT_PATH, sim_type = spatial_activity_type, init_condition = INIT_SHAPE, 
                            activity= activity_parameters)

## Filament parameters (common)

In [3]:
#-------------------------------------------------------------
# Filament parameters (common to all kinds of simulations)
#-------------------------------------------------------------
# No:of colloids
NP = 32 
# Potential Dipole strength
D0 = 1.5 

N_CYCLES = 50

DELTA_T = 5 # This is roughly 4X the axial stretch time-scale which is the smallest time-scale in the dynamics

# Relative duration of compression to total cycle duration
DUTY_CYCLE = 0.5 

# Start phase of the activity cycle
START_PHASE = 0

# Default activity timescale
activity_timescale = 750

# Default simulation time
Tf = activity_timescale*N_CYCLES
Npts = int(Tf/DELTA_T) # No:of time points saved

# Spatial activity type
spatial_activity_type = 'lacry' # Spatial activity distribution type

assert spatial_activity_type in SPATIAL_ACTIVITY_TYPES, 'Spatial Activity type not supported'

# Temporal activity type
temporal_activity_type = 'square-wave' # Choose from 'square-wave', 'biphasic', 'normal' and 'lognormal'

assert temporal_activity_type in TEMPORAL_ACTIVITY_TYPES , 'Activity type not supported'



# Run one of the cells below (A-D) to define a temporal activity profile for the simulations

## (A) Deterministic/Square-wave activity

In [4]:
# Square-wave (deterministic) activity (Uncomment below)
# activity_parameters = {'type':'square-wave','activity time scale':activity_timescale, 'duty_cycle':DUTY_CYCLE, 
#                         'start phase':START_PHASE}

# Constant compressive activity
DUTY_CYCLE = 1.0

activity_parameters = {'type':'square-wave','activity time scale':activity_timescale, 'duty_cycle':DUTY_CYCLE, 
                        'start phase':START_PHASE}

## (B) Biphasic activity

In [ ]:
# Biphasic activity (Uncomment below)
slow_timescale = 750
fast_timescale = 250
n_slow_cycles = 1
n_fast_cycles = 3
n_activity_cycles = 10 # No:of activity cycles we want to simulate (for biphasic activity this is the number of slow and fast cycles)
Tf = n_activity_cycles*(n_slow_cycles*slow_timescale+n_fast_cycles*fast_timescale)
time_step_save = 5 # This is roughly 4X the axial screate_save_foldertretch time-scale which is the smallest time-scale in the dynamics
Npts = int(Tf/time_step_save) # No:of time points saved
activity_parameters = {'type':'biphasic','activity time scale':{'slow':slow_timescale, 'fast':fast_timescale}, 
                       'duty_cycle':0.5,'N_cycles':{'slow':n_slow_cycles, 'fast':n_fast_cycles}, 
                       'start_state':'slow', 'start phase':START_PHASE}

## (C) Lognormal activity

In [26]:
# Lognormal distributed activity times
activity_parameters = {'type':'lognormal','activity time scale':activity_timescale,
                    'sigma extension':SIGMA_EXT, 'sigma compression':SIGMA_COMP, 
                    'n_cycles':N_CYCLES}

## (D) Normally distributed activity

In [ ]:
# Normally distributed activity times
activity_parameters={'type':'normal','activity time scale':activity_timescale, 
'n_cycles': N_CYCLES, 'duty_cycle':DUTY_CYCLE, 'noise_scale':0.20}

## Run a Batch Simulation

In [5]:
# Setup and run parallel simulation batch
parameter_to_vary = 'D0'

assert parameter_to_vary in PARAMETERS, "Not a valid parameter for filament simulations"

parameter_values = np.linspace(0.1, 0.75, 10)
# parameter_values = [100, 200]

N_IC = 1 # No:of initial conditions to simulate at a given parameter value

parameter_list_full = []

for ii in range(N_IC):
    parameter_list_full.append(parameter_values)

parameter_list_full = np.array(parameter_list_full).flatten()

print(parameter_list_full)

num_cores = multiprocessing.cpu_count()

print("No:of detected cores: {}".format(num_cores))

results = Parallel(n_jobs = num_cores, verbose=10) \
        (delayed(run_parametric_simulation)(parameter_to_vary, value, activity_parameters, Tf, Npts, spatial_activity_type) for pid, value in enumerate(parameter_list_full))

[0.1        0.17222222 0.24444444 0.31666667 0.38888889 0.46111111
 0.53333333 0.60555556 0.67777778 0.75      ]
No:of detected cores: 16


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   3 out of  10 | elapsed:  2.2min remaining:  5.2min
[Parallel(n_jobs=16)]: Done   5 out of  10 | elapsed:  2.3min remaining:  2.3min
[Parallel(n_jobs=16)]: Done   7 out of  10 | elapsed:  2.5min remaining:  1.1min
[Parallel(n_jobs=16)]: Done  10 out of  10 | elapsed:  3.0min finished


In [3]:
INIT_SHAPE

{'shape': 'line'}